## Overview

Kamodo provides a *functional* interface for space weather analysis, visualization, and knowledge discovery, allowing many problems in scientific data analysis to be posed in terms of function composition and evaluation. We'll walk through its general features here.

##  Kamodo objects

Users primarily interact with models and data through Kamodo objects. 

In [1]:
from kamodo import Kamodo

### Function registration
Kamodo objects are essentially python dictionaries storing variable symbols as keys and their interpolating functions as values. New functions may be registered either at the initialization of the Kamodo object or later using dictionary bracket syntax.

In [2]:
kamodo = Kamodo(f = 'x**2')
kamodo

{f(x): <function _lambdifygenerated at 0x10f9b47c0>, f: <function _lambdifygenerated at 0x10f9b47c0>}

In [3]:
kamodo.f(3)

9

In [4]:
kamodo = Kamodo('$x = t^2$')
kamodo['g'] = 'y-1'
kamodo

{x(t): <function _lambdifygenerated at 0x13ee21800>, x: <function _lambdifygenerated at 0x13ee21800>, g(y): <function _lambdifygenerated at 0x13ee21c60>, g: <function _lambdifygenerated at 0x13ee21c60>}

In [5]:
kamodo.g(3)

2

### Function composition
Kamodo automatically composes functions through specifying on the right-hand-side.

In [6]:
kamodo['f'] = 'g(x)'
kamodo

{x(t): <function _lambdifygenerated at 0x13ee21800>, x: <function _lambdifygenerated at 0x13ee21800>, g(y): <function _lambdifygenerated at 0x13ee21c60>, g: <function _lambdifygenerated at 0x13ee21c60>, f(t): <function _lambdifygenerated at 0x13ee228e0>, f: <function _lambdifygenerated at 0x13ee228e0>}

Here we have defined two functions $x(t)$, $g(y)$, and the composition $g∘f$. Kamodo was able to determine that $f$ is implicitly a function of $t$ even though we did not say so in $f$'s declaration.

#### Function evaluation
Kamodo uses sympy's ```lambdify``` function to turn the above equations into highly optimized functions for numerical evaluation. We may evaluate $f(t)$ for $t=3$ using "dot" notation:

In [7]:
kamodo.f(3)

8

In [8]:
help(kamodo.g)

Help on function _lambdifygenerated:

_lambdifygenerated(y)
    Created with lambdify. Signature:
    
    func(y)
    
    Expression:
    
    y - 1
    
    Source code:
    
    def _lambdifygenerated(y):
        return (y - 1)
    
    
    Imported modules:



where the return type is a numpy array. We could also have passed in a numpy array and the result shares the same shape:

In [9]:
import numpy as np
t = np.linspace(-5, 5, 100000)
result = kamodo.f(t)

In [10]:
assert(t.shape == result.shape)

### Unit conversion
Kamodo automatically handles unit conversions. Simply declare units on the left-hand-side of expressions using bracket notation.

In [11]:
kamodo = Kamodo('mass[kg] = x', 'vol[m^3] = y')

In [12]:
kamodo

{mass(x): <function _lambdifygenerated at 0x13ec6c720>, mass: <function _lambdifygenerated at 0x13ec6c720>, vol(y): <function _lambdifygenerated at 0x13efd4180>, vol: <function _lambdifygenerated at 0x13efd4180>}

Unless specified, Kamodo will assign the units for newly defined variables:

In [13]:
kamodo['rho'] = 'mass/vol'
kamodo

{mass(x): <function _lambdifygenerated at 0x13ec6c720>, mass: <function _lambdifygenerated at 0x13ec6c720>, vol(y): <function _lambdifygenerated at 0x13efd4180>, vol: <function _lambdifygenerated at 0x13efd4180>, rho(x, y): <function _lambdifygenerated at 0x13ee23ba0>, rho: <function _lambdifygenerated at 0x13ee23ba0>}

We may override the default behavior by simply naming the our chosen units in the left hand side.

In [14]:
kamodo['rho(x,y)[g/cm^3]'] = 'mass/vol'
kamodo

{mass(x): <function _lambdifygenerated at 0x13ec6c720>, mass: <function _lambdifygenerated at 0x13ec6c720>, vol(y): <function _lambdifygenerated at 0x13efd4180>, vol: <function _lambdifygenerated at 0x13efd4180>, rho(x, y): <function _lambdifygenerated at 0x13ee23ba0>, rho: <function _lambdifygenerated at 0x13ee23ba0>, rho(x, y): <function _lambdifygenerated at 0x13ec6d3a0>, rho: <function _lambdifygenerated at 0x13ec6d3a0>}

!!! note
    Kamodo will raise an error if the left and right-hand-side units are incompatible.

Even though generated functions are unitless, the units are clearly displayed on the lhs. We think this is a good trade-off between performance and legibility.

In [15]:
kamodo.detail()

,symbol,units,lhs,rhs,arg_units
mass,mass(x),kg,mass,x,{}
vol,vol(y),m**3,vol,y,{}
rho,"rho(x, y)",g/cm**3,"rho(x, y)",mass(x)/(1000*vol(y)),"{'x': 'cm**(-3)', 'y': 'g'}"


In [16]:
kamodo.rho(3,4)

0.00075

We can verify that kamodo produces the correct output upon evaluation.

In [17]:
assert(kamodo.rho(3,8) == (3*1000.)/(8*100**3))

### Variable naming conventions

Kamodo allows for a wide array of variable names to suite your problem space, including greek, subscripts, superscripts.

In [18]:
kamodo = Kamodo(
    'rho = ALPHA+BETA+GAMMA',
    'rvec = t',
    'fprime = x',
    'xvec_i = xvec_iminus1 + 1',
    'F__gravity = G*M*m/R**2',
)
kamodo

{rho(ALPHA, BETA, GAMMA): <function _lambdifygenerated at 0x13efd7ce0>, rho: <function _lambdifygenerated at 0x13efd7ce0>, rvec(t): <function _lambdifygenerated at 0x13f0284a0>, rvec: <function _lambdifygenerated at 0x13f0284a0>, fprime(x): <function _lambdifygenerated at 0x13f028860>, fprime: <function _lambdifygenerated at 0x13f028860>, xvec_i(xvec_iminus1): <function _lambdifygenerated at 0x13f028c20>, xvec_i: <function _lambdifygenerated at 0x13f028c20>, F__gravity(G, M, R, m): <function _lambdifygenerated at 0x13ee23b00>, F__gravity: <function _lambdifygenerated at 0x13ee23b00>}

For more details on variable naming conventions, see the [Syntax](../Syntax/) section.

### Lambda functions

Kamodo borrows heavily from functional programing techniques, including use of the ["lambda" function](https://en.wikipedia.org/wiki/Anonymous_function), an abstract representation of a function. Think of the lambda as a placeholder when no closed-form expression may be given for a registered function.

In the example below, the function `rho` contains an implementation which may be hidden from the end user. In this case, it simply hands the argument off to another library.

In [19]:
def rho(x):
    return np.sin(x)

When registering the above function as part of a Kamodo object, Kamodo will assign the greek letter $\rho$ to the left-hand-side following the [Syntax Conventions](../Syntax/). For the right hand side the generic $\lambda(x)$ is used to signify that no [closed-form expression](https://en.wikipedia.org/wiki/Closed-form_expression) is available.

In [20]:
Kamodo(rho = rho)

{rho(x): <function rho at 0x13f0298a0>, rho: <function rho at 0x13f0298a0>}

If a function author wishes to provide a placeholder expression for the right-hand-side, they may do so using the `@kamodofy` decorator below.

## @kamodofy Decorator

Many functions can not be written as simple mathematical expressions - they could represent simulation output or observational data. For this reason, we provide a ```@kamodofy``` decorator, which turns any callable function into a kamodo-compatible variable and adds metadata that enables unit conversion.

In [21]:
from kamodo import kamodofy, Kamodo
import numpy as np

@kamodofy(units = 'kg/m**3', citation = 'Bob et. al, 2018', equation='x+y') 
def rho(x = np.array([3,4,5]), y = np.array([1,2,3])):
    """A function that computes density"""
    return x+y

kamodo = Kamodo(rho = rho)
kamodo['den[g/cm^3]'] = 'rho'
kamodo

{rho(x, y): <function rho at 0x13f02a160>, rho: <function rho at 0x13f02a160>, den(x, y): <function _lambdifygenerated at 0x13f029440>, den: <function _lambdifygenerated at 0x13f029440>}

In [22]:
kamodo.den(3,4)

0.007

In [23]:
kamodo.rho.meta # PyHC standard

{'units': 'kg/m**3',
 'arg_units': None,
 'citation': 'Bob et. al, 2018',
 'equation': 'x+y',
 'hidden_args': []}

In [24]:
kamodo.rho()

array([4, 6, 8])

In [25]:
kamodo.rho.data # PyHC standard

array([4, 6, 8])

Original function doc strings and signatures passed through

In [26]:
help(kamodo.rho)

Help on function rho in module __main__:

rho(x=array([3, 4, 5]), y=array([1, 2, 3]))
    A function that computes density
    
    citation: Bob et. al, 2018



In [27]:
kamodo.detail()

,symbol,units,lhs,rhs,arg_units
rho,"rho(x, y)",kg/m**3,rho,x+y,None
den,"den(x, y)",g/cm**3,den,"rho(x, y)/1000",{}


# Visualization

Kamodo graphs are generated directly from function signatures by examining the structure of both output and input arguments. 

In [28]:
from plotting import plot_types
plot_types

plot_type  \
out_shape arg_shapes                                       
(1,)      ((N, M), (N, M), (N, M))         3d-parametric   
(N,)      ((N,),)                                1d-line   
          ((N,), (N,))                    2d-line-scalar   
          ((N,), (N,), (N,))              3d-line-scalar   
          ((N, 3),)                           3d scatter   
(N, 2)    ((N,),)                                2d-line   
          ((N, 2),)                            2d-vector   
(N, 3)    ((N,),)                                3d-line   
          ((N, 3),)                            3d-vector   
          ((M,), (M,), (M,))              3d-tri-surface   
(N, N)    ((N,), (N,))                        2d-contour   
(N, M)    ((N,), (M,))                        2d-contour   
          ((M,), (N,))                        2d-contour   
          ((N, M), (N, M))               2d-contour-skew   
          ((N, M), (N, M), (N, M))  3d-parametric-scalar   
          ((1,), (N, M), (N, M))                3d-plane   
          ((N, M), (1,), (N, M))                3d-plane   
          ((N, M), (N, M), (1,))                3d-plane   
(N, 1, M) ((N,), (1,), (M,))                    3d-plane   
          ((1,), (N,), (M,))                    3d-plane   
(1, N, M) ((N,), (1,), (M,))                    3d-plane   
(N, M, 1) ((1,), (N,), (M,))                    3d-plane   
          ((N,), (1,), (M,))                    3d-plane   
          ((N,), (M,), (1,))                    3d-plane   
          ((M,), (N,), (1,))                    3d-plane   
(N, M, 3) ((N,), (M,))                             image   

                                                     function  
out_shape arg_shapes                                           
(1,)      ((N, M), (N, M), (N, M))           plotting.surface  
(N,)      ((N,),)                          plotting.line_plot  
          ((N,), (N,))                     plotting.line_plot  
          ((N,), (N,), (N,))               plotting.line_plot  
          ((N, 3),)                     plotting.scatter_plot  
(N, 2)    ((N,),)                          plotting.line_plot  
          ((N, 2),)                      plotting.vector_plot  
(N, 3)    ((N,),)                          plotting.line_plot  
          ((N, 3),)                      plotting.vector_plot  
          ((M,), (M,), (M,))        plotting.tri_surface_plot  
(N, N)    ((N,), (N,))                  plotting.contour_plot  
(N, M)    ((N,), (M,))                  plotting.contour_plot  
          ((M,), (N,))                  plotting.contour_plot  
          ((N, M), (N, M))              plotting.contour_plot  
          ((N, M), (N, M), (N, M))           plotting.surface  
          ((1,), (N, M), (N, M))               plotting.plane  
          ((N, M), (1,), (N, M))               plotting.plane  
          ((N, M), (N, M), (1,))               plotting.plane  
(N, 1, M) ((N,), (1,), (M,))                   plotting.plane  
          ((1,), (N,), (M,))                   plotting.plane  
(1, N, M) ((N,), (1,), (M,))                   plotting.plane  
(N, M, 1) ((1,), (N,), (M,))                   plotting.plane  
          ((N,), (1,), (M,))                   plotting.plane  
          ((N,), (M,), (1,))                   plotting.plane  
          ((M,), (N,), (1,))                   plotting.plane  
(N, M, 3) ((N,), (M,))                         plotting.image

Kamodo uses [plotly](https://plot.ly/python/) for visualization, enabling a rich array of interactive graphs and easy web deployment.

In [29]:
import plotly.io as pio
from plotly.offline import iplot,plot, init_notebook_mode
init_notebook_mode(connected=True)

In [30]:
@kamodofy(units = 'kg/m^3')
def rho(x = np.linspace(0,1, 20), y = np.linspace(-1, 1, 40)):
    """A function that computes density"""
    x_, y_ = np.meshgrid(x,y)
    return x_*y_

kamodo = Kamodo(rho = rho)
kamodo

{rho(x, y): <function rho at 0x13fa176a0>, rho: <function rho at 0x13fa176a0>}

We will generate an image of this function using plotly

In [31]:
fig = kamodo.plot('rho')
# pio.write_image(fig, 'images/Kamodo_fig1.svg')

![fig1](images/Kamodo_fig1.svg)

See the [Visualization](../Visualization/) section for detailed examples.

## Latex I/O

Even though math is the language of physics, most scientific analysis software requires you to learn new programing languages. Kamodo allows users to write their mathematical expressions in LaTeX, a typesetting language most scientists already know:

In [32]:
kamodo = Kamodo('$rho[kg/m^3] = x^3$', '$v[cm/s] = y^2$')
kamodo['p[Pa]'] = '$\\rho v^2$'
kamodo

{rho(x): <function _lambdifygenerated at 0x1400afd80>, rho: <function _lambdifygenerated at 0x1400afd80>, v(y): <function _lambdifygenerated at 0x1400afe20>, v: <function _lambdifygenerated at 0x1400afe20>, p(x, y): <function _lambdifygenerated at 0x1400af380>, p: <function _lambdifygenerated at 0x1400af380>}

The resulting equation set may also be exported as a LaTeX string for use in publications:

In [33]:
print(kamodo.to_latex() + '\n.')

\begin{equation}\rho{\left(x \right)}[\frac{kg}{m^{3}}] = x^{3}\end{equation} \begin{equation}v{\left(y \right)}[cm / s] = y^{2}\end{equation} \begin{equation}p{\left(x,y \right)}[Pa] = \frac{\rho{\left(x \right)} v^{2}{\left(y \right)}}{10000}\end{equation}
.


# Simulation api

Kamodo offers a simple api for functions composed of each other.

Define variables as usual (order matters).

In [34]:
kamodo = Kamodo()
kamodo['y_iplus1'] = 'x_i + 1'
kamodo['x_iplus1'] = 'y_i - 2'
kamodo

{y_iplus1(x_i): <function _lambdifygenerated at 0x1400afb00>, y_iplus1: <function _lambdifygenerated at 0x1400afb00>, x_iplus1(y_i): <function _lambdifygenerated at 0x140125120>, x_iplus1: <function _lambdifygenerated at 0x140125120>}

Now add the ```update``` attribute to map functions onto arguments.

In [35]:
kamodo.x_iplus1.update = 'x_i'
kamodo.y_iplus1.update = 'y_i'

Create a simulation with initial conditions

In [36]:
simulation = kamodo.simulate(x_i = 0, steps = 5)
simulation #an iterator of arg, val dictionaries

<generator object simulate at 0x13ff26c00>

Run the simulation by iterating through the generator.

In [37]:
import pandas as pd
pd.DataFrame(simulation) # pandas conveniently iterates the results for display

,y_i,x_i
0,NaN,0
1,1.0,-1
2,0.0,-2
3,-1.0,-3
4,-2.0,-4
5,-3.0,-5
